<a href="https://colab.research.google.com/github/OmdenaAI/botswana-chapter-desertification-detection/blob/main/ndvi_visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install geehydro
!pip install ipygee
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import ee
import folium 
import geehydro
import numpy as np # yes, numpy!
import pandas as pd # yes, pandas!
import matplotlib.pyplot as plt 
from ipygee import*
from pandas.plotting import register_matplotlib_converters
import geemap

ee.Initialize()


In [6]:
wdpa_polygon = ee.FeatureCollection('WCMC/WDPA/current/polygons')
# select a national park region shapefile
national_park = wdpa_polygon.filter(ee.Filter.eq('NAME', 'Chobe'))

Map = folium.Map(location=[-22.225292, 23.800886], zoom_start=8)
Map.addLayer(national_park)
Map



In [19]:
# Select modis image collection to get NDVI
modis = ee.ImageCollection('MODIS/006/MOD13Q1')

min_date = '2000-01-01'
max_date = '2020-12-01'

modis = modis.filterDate(ee.DateRange(min_date, max_date))

available_dates = modis.aggregate_array('system:time_start').map(
    lambda d: ee.Date(d).format('YYYY-MM-dd')
).getInfo()
print("available dates, from {} to {}".format(available_dates[0], available_dates[-1]))

# select EVI and NDVI
evi = modis.select('EVI')
ndvi = modis.select('NDVI')
def scale_factor(image):
  # scale factor for the MODIS MOD13Q1 product
  return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

# mapping function to multiply by the scale factor
scaled_evi = evi.map(scale_factor)
scaled_ndvi = ndvi.map(scale_factor)
Map.addLayer(scaled_ndvi.mean().clip(national_park),
                  vis_params={'min': -1,
                              'max': 1,
                              'palette': ['red', 'yellow','green']})
Map.centerObject(national_park)

Map

available dates, from 2000-02-18 to 2020-11-16


Map(bottom=144976.7568359375, center=[-18.36072560399234, 26.1435878276825], controls=(WidgetControl(options=[…

In [44]:
selected_roi_ndvi = chart.Image.series(**{'imageCollection': scaled_ndvi,
                                   'region': national_park,
                                   'reducer': ee.Reducer.mean(),
                                   'scale': 1000,
                                   'xProperty': 'system:time_start'})
selected_roi_ndvi.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [41]:
# Timeseries data can be extracted as a Panda's dataframe 
selected_roi_ndvi.dataframe

,NDVI
2000-02-18,0.457243
2000-03-05,0.485756
2000-03-21,0.487298
2000-04-06,0.468957
2000-04-22,0.435049
...,...
2020-09-13,0.210298
2020-09-29,0.206939
2020-10-15,0.223215
2020-10-31,0.271948


Experiment to calculate an display NDWI index next to Modis true color source

In [27]:
SELECTED_NDVI_MIN_VALUE = 0.1
SELECTED_NDVI_MAX_VALUE = 0.9

modis_truecolor = ee.ImageCollection('MODIS/006/MCD43A4')
modis_truecolor = modis_truecolor.filterDate(ee.DateRange(min_date, max_date))


def select_ndvi_band(img):
    ndwi_image = img.normalizedDifference(['Nadir_Reflectance_Band1', 
                                           'Nadir_Reflectance_Band3'])
    #m = ndwi_image.lt(SELECTED_NDVI_MIN_VALUE).And(ndwi_image.lt(SELECTED_NDVI_MAX_VALUE))
    return ndwi_image

scaled_ndvi_images = modis_truecolor.map(select_ndvi_band)
ndwi_masks = scaled_ndvi_images.map(lambda img: img.selfMask())

available_dates = modis_truecolor.aggregate_array('system:time_start').map(
    lambda d: ee.Date(d).format('YYYY-MM-dd')
).getInfo()

ndwi_layer_names = ['NDWI ' + str(date) for date in available_dates]
modis_layer_names = ['Modis ' + str(date) for date in available_dates]

ndwi_mask_vis = {'palette': 'hot'}

ndwi_vis = {
    'min': -1,
    'max': 1
}

modis_vis = {
    'bands': ['Nadir_Reflectance_Band1', 
              'Nadir_Reflectance_Band4',
              'Nadir_Reflectance_Band3'],
    'min': 0,
    'max': 4000,
    'gamma': 1.4
}

Map = geemap.Map()
Map.ts_inspector(
    left_ts=ndwi_masks,
    right_ts=modis_truecolor,
    left_names=ndwi_layer_names,
    right_names=modis_layer_names,
    left_vis=ndwi_vis,
    right_vis=modis_vis,
)
Map.centerObject(national_park, zoom=8)
Map

Map(center=[-18.517907370005872, 24.50724143690363], controls=(WidgetControl(options=['position', 'transparent…